In [1]:
from neo4j import GraphDatabase

In [2]:
# Neo4j connection settings (replace with your credentials)
URI = "bolt://localhost:7687"
USER = "neo4j"
PASSWORD = "neo4j" # Default is "neo4j" (change after first login)

# Sample data
users = [
{"id": 0, "name": "Hero", "age": 20, "interests": ["music", "movies"]},
{"id": 1, "name": "Dunn", "age": 25, "interests": ["sports"]},
{"id": 2, "name": "Sue"},
{"id": 3, "name": "Chi"},
{"id": 4, "name": "Thor"},
{"id": 5, "name": "Clive"},
{"id": 6, "name": "Hicks"},
{"id": 7, "name": "Devin"},
{"id": 8, "name": "Kate"},
{"id": 9, "name": "Klein"}
]

friendship_pairs = [(0, 1), (0, 2), (1, 2), (1, 3), (2, 3), (3, 4),
(4, 5), (5, 6), (5, 7), (6, 8), (7, 8), (8, 9)]

In [3]:
# New relationships: "LIKES" and "FOLLOWS"
relationships = [
    {"type": "FRIENDS_WITH", "pairs": friendship_pairs},
    {"type": "LIKES", "pairs": [(0, "music"), (1, "sports"), (2, "movies")]},
    {"type": "FOLLOWS", "pairs": [(0, 3), (1, 4), (2, 5)]}
]

In [4]:
def create_advanced_graph(tx):
    # Clear existing data
    tx.run("MATCH (n) DETACH DELETE n")

    # Create User nodes with properties
    for user in users:
        # Added this code to handle optional/missing fields
        props = {
            "id": user["id"],
            "name": user["name"],
            "age": user.get("age"),  # Returns None if 'age' doesn't exist
            "interests": user.get("interests", [])  # Deafult to empty list
        }    
        tx.run(
            """
            CREATE (u:User {id: $id, name: $name, age: $age, interests: $interests})
            """,
            ## id=user["id"], name=user["name"], age=user["age"], interests=user["interests"]
            **props 
        )
    
    # Create relationships
    for rel in relationships:
        for pair in rel["pairs"]:
            if rel["type"] == "LIKES":
                # Link users to interests (treated as nodes)
                tx.run(
                    """
                    MERGE (u:User {id: $id})
                    MERGE (i:Interest {name: $interest})
                    CREATE (u)-[:LIKES]->(i)
                    """,
                    id=pair[0], interest=pair[1]
                )
            else:
                # User-to-user relationships (FRIENDS_WITH, FOLLOWS)
                tx.run(
                    """
                    MATCH (u1:User {id: $id1}), (u2:User {id: $id2})
                    CREATE (u1)-[:%s]->(u2)
                    """ % rel["type"],
                    id1=pair[0], id2=pair[1]
                )

# Execute
with GraphDatabase.driver(URI, auth=(USER, PASSWORD)) as driver:
    with driver.session() as session:
        session.execute_write(create_advanced_graph)
    print("Advanced graph populated!")

Advanced graph populated!


In [5]:
# I am showing the screenshotS of database created - run this cellS to see the pictureS
from PIL import Image

img = Image.open("4_Users and interests.jpg")
img.show()        # displays the image through the built-in Photo app in your respective OS.

In [6]:
img = Image.open("4_FOLLOWS.jpg")
img.show() 

In [7]:
img = Image.open("4_LIKES.jpg")
img.show() 